In [ ]:
library(lavaan)

In [ ]:
experiment.data <- read.csv('data/widefmt-experiment-data.csv')

In [ ]:
experiment.data$E2 <- experiment.data$E2 * -1
experiment.data$Q4 <- experiment.data$Q4 * -1

## 1. Confirmatory Factor Analysis

In [ ]:
fit.measures = c("chisq", "df", "pvalue", "rmsea", "rmsea.ci.lower", "rmsea.ci.upper", "cfi", "tli")

In [ ]:
construct_reliability <- function(df){
    #' Average Variance Extracted (AVE)
    #'   The grand mean value of the squared loadings of the indicators
    #'   associated with a particular construct (i.e., the sum of the squared
    #'   loadings divided by the numbers of indicators). (Hair et al., 2017; pp. 130)
    df <- df[df$op == "=~",]

    result <- NULL
    
    factors <- unique(df$lhs)
    for (factor in factors) {
        std_loadings <- df[df$lhs == factor,][["est.std"]]
        squared_loadings <- std_loadings^2

        # Average Variance Extracted (AVE)
        ave <- mean(squared_loadings)
        sqrt_ave <- round(sqrt(ave), digits = 3)
        ave <- round(ave, digits = 3)

        # Composite Reliability (CR)
        sum_loadings <- sum(std_loadings)
        var_e <- 1 - squared_loadings
        
        comp_r <- (sum_loadings^2) / ((sum_loadings^2) + sum(var_e))
        comp_r <- round(comp_r, digits = 3)

        res <- c(factor, comp_r, ave, sqrt_ave)
        res <- array(res, dim=c(1, length(res)))

        result <- if (is.null(result)) res else rbind(result, res)
    }

    result <- data.frame(result)
    colnames(result) <- c("factor", "comp_r", "ave", "sqrt_ave")

    return(result)
}

### 1.1 Original Model

In [ ]:
measurement.model <- ' effctv =~ E1 + E2 + E3
trust =~ T1 + T2 + T3 + T4
dvrsty =~ D1 + D2 + D3
qualty =~ Q1 + Q2 + Q3 + Q4
undrst =~ P1 + P2 + P3 '

In [ ]:
measurement.model.fit <- cfa(measurement.model,
                             data = experiment.data,
                             ordered = c("E1", "E2", "E3",
                                         "T1", "T2", "T3", "T4",
                                         "D1", "D2", "D3",
                                         "Q1", "Q2", "Q3", "Q4",
                                         "P1", "P2", "P3"),
                             estimator = "DWLS")

In [ ]:
estimates <- standardizedSolution(measurement.model.fit)

In [ ]:
print(estimates[estimates$op == "=~",])

In [ ]:
print(construct_reliability(estimates))

In [ ]:
print(lavInspect(measurement.model.fit, what = "cor.lv"))

🔍 __What should we look at?__
1. Comparative Fit Index (CFI) and Tucker-Lewis Index (TLI) have to be values greater than 0.95 (Hu and Bentler, 1999, as cited in Xia and Yang, 2018).
2. Root Mean Squared Error of Approximation (RMSEA) smaller than 0.06 suggests relatively good model–data fit (Hu and Bentler, 1999, as cited in Xia and Yang, 2018).
3. A common rule of thumb is that (standardized) loadings should be 0.708 or higher, since that number squared (i.e., communality) is 0.5 (Hair et al., 2017, pp. 128).
4. Generally, if the loading of an indicator is below 0.4, remove it; if it's greater than or equal to 0.4 and less than 0.7, remove it if, by doing so, there is an increase in composite reliability or AVE above the suggested threshold value (Hair et al., 2017, pp. 128).
5. Factors with less than three indicators should be discarded from further analysis (Knijnenburg and Willemsen, 2015).
6. In the table of estimates provided by `lavaan`, the standardized factor loadings are in the `Std.all` column. At the row level, focus on the `Latent Variables` section.
7. Composite reliability and Cronbach's alpha: values below 0.6 denote a lack of internal consistency reliability, values in the 0.6–0.7 range are acceptable in exploratory research; and values in the 0.7–0.9 range in later stages of research are deemed satisfactory; values above 0.90 are not desirable (Hair et al., 2017).
8. Discriminant Validity (Fronell-Larcker criterion): The square root of each construct's AVE should be greater than its highest correlation with any other construct (Hair et al., 2017; Knijnenburg and Willemsen, 2015).
9. An alternative approach to evaluating the results of the Fornell-Larcker criterion is to determine whether the AVE is larger than the squared correlation with any other construct (Hair et al., 2017).

References:
1. Curran, T. (May, 2019). [Exploratory and Confirmatory Factor Anaysis (LT3)](https://www.thomascurran.co.uk/courses/pb230/lt3/)
2. Hair, J., Hult, G., Ringle, C., & Sarstedt, M. (2017). [A Primer on Partial Least Squares Structural Equation Modeling (PLS-SEM)](https://eli.johogo.com/Class/CCU/SEM/_A%20Primer%20on%20Partial%20Least%20Squares%20Structural%20Equation%20Modeling_Hair.pdf) (2nd ed.).
3. Knijnenburg, B., & Willemsen, M. (2015). [Evaluating recommender systems with user experiments](https://link.springer.com/chapter/10.1007/978-1-4899-7637-6_9).

### 1.2 First Iteration

In [ ]:
measurement.model2 <- ' effctv =~ E1 + E2 + E3
trust =~ T1 + T2 + T3 + T4
qualty =~ Q1 + Q2 + Q3 + Q4
undrst =~ P1 + P2 + P3 '

In [ ]:
measurement.model2.fit <- cfa(measurement.model2,
                              data = experiment.data,
                              ordered = c("E1", "E2", "E3",
                                          "T1", "T2", "T3", "T4",
                                          "Q1", "Q2", "Q3", "Q4",
                                          "P1", "P2", "P3"),
                              estimator="DWLS")

In [ ]:
estimates <- standardizedSolution(measurement.model2.fit)

In [ ]:
print(construct_reliability(estimates))

In [ ]:
print(lavInspect(measurement.model2.fit, what = "cor.lv"))

### 1.3 Second Iteration

In [ ]:
measurement.model3 <- ' effctv =~ E1 + E2 + E3
trust =~ T1 + T2 + T3
qualty =~ Q1 + Q2 + Q3
undrst =~ P1 + P2 + P3 '

In [ ]:
measurement.model3.fit <- cfa(measurement.model3,
                              data = experiment.data,
                              ordered = c("E1", "E2", "E3",
                                          "T1", "T2", "T3",
                                          "Q1", "Q2", "Q3",
                                          "P1", "P2", "P3"),
                              estimator="DWLS")

In [ ]:
print(fitMeasures(measurement.model3.fit, fit.measures))

In [ ]:
estimates <- standardizedSolution(measurement.model3.fit)

In [ ]:
print(construct_reliability(estimates))

In [ ]:
print(lavInspect(measurement.model3.fit, what = "cor.lv"))

In [ ]:
print(estimates[estimates$op == "=~",])

### 1.4 Third Iteration

In [ ]:
measurement.model4 <- ' effctv =~ E1 + E2 + E3
qualty =~ Q1 + Q2 + Q3
undrst =~ P1 + P2 + P3 '

In [ ]:
measurement.model4.fit <- cfa(measurement.model4,
                              data = experiment.data,
                              ordered = c("E1", "E2", "E3",
                                          "Q1", "Q2", "Q3",
                                          "P1", "P2", "P3"),
                              estimator="DWLS")

In [ ]:
print(fitMeasures(measurement.model4.fit, fit.measures))

In [ ]:
estimates <- standardizedSolution(measurement.model4.fit)

In [ ]:
print(construct_reliability(estimates))

In [ ]:
print(lavInspect(measurement.model4.fit, what = "cor.lv"))

In [ ]:
print(estimates[estimates$op == "=~",])

## 2. Structural Equation Modelling

👉 __Structural equation modeling with ordered categorical data__
1. To address the categorical nature of data, the diagonally weighted least squares (DWLS) estimator based on the polychoric correlation matrix has become the most popular method (Savalei & Rhemtulla, 2013, as cited in Xia and Yang, 2018).
2. Given that the chi-square fit statistic is affected by large samples, the ratio of the chi-square statistic to the respective degrees of freedom is preferred. A ratio of ≤ 2 indicates a superior fit between the hypothesized model and the sample data (Cole, 1987, as cited in Alavi et al., 2020).
3. A non-significant χ2 value (_p_ > .05) is indicative of a path model that fits the data well (Shah, 2012).

__Useful links__
1. [Chi Square p-value in SEM explained](https://www.youtube.com/watch?v=SHwBTRFGaYQ)
2. [Non-significant result in CFA model](https://stats.stackexchange.com/a/442059)
3. [Goodness of fit](https://stats.stackexchange.com/a/439656)

References:
1. Alavi, M., Visentin, D. C., Thapa, D. K., Hunt, G. E., Watson, R., & Cleary, M. L. (2020). [Chi-square for model fit in confirmatory factor analysis](https://onlinelibrary.wiley.com/doi/full/10.1111/jan.14399).
2. Shah, R. B. (2012). [A multivariate analysis technique: Structural equation modeling](https://tarj.in/wp-content/uploads/paper/AJMR/2012/AJMR-SEPTEMBER-2012.pdf#page=57).
3. Xia, Y., & Yang, Y. (2019). [RMSEA, CFI, and TLI in structural equation modeling with ordered categorical data: The story they tell depends on the estimation methods](https://link.springer.com/article/10.3758/s13428-018-1055-2).

### 2.1 Saturated Model

In [ ]:
saturated.path.model <- ' effctv =~ E1 + E2 + E3
qualty =~ Q1 + Q2 + Q3
undrst =~ P1 + P2 + P3
effctv ~ qualty + undrst + ctrlclin + ctrlhetg
qualty ~ undrst + ctrlclin + ctrlhetg
undrst ~ ctrlclin + ctrlhetg '

In [ ]:
saturated.path.model.fit <- sem(saturated.path.model,
                                data = experiment.data,
                                ordered=c("E1", "E2", "E3",
                                          "Q1", "Q2", "Q3",
                                          "P1", "P2", "P3"),
                                estimator="DWLS")

In [ ]:
estimates <- standardizedSolution(saturated.path.model.fit)

In [ ]:
print(estimates[estimates$op == "~",])

### 2.2 Pruned Model

In [ ]:
pruned.path.model <- ' effctv =~ E1 + E2 + E3
qualty =~ Q1 + Q2 + Q3
undrst =~ P1 + P2 + P3
effctv ~ c*qualty
qualty ~ b*undrst
undrst ~ a*ctrlhetg
ab := a*b
bc := b*c
abc := a*b*c '

In [ ]:
pruned.path.model.fit <- sem(pruned.path.model,
                             data = experiment.data,
                             ordered=c("E1", "E2", "E3",
                                       "Q1", "Q2", "Q3",
                                       "P1", "P2", "P3"),
                             estimator="DWLS")

In [ ]:
print(fitMeasures(pruned.path.model.fit, fit.measures))

In [ ]:
estimates <- standardizedSolution(pruned.path.model.fit)

In [ ]:
print(estimates[estimates$op %in% c("~", ":="),])

In [ ]:
print(lavInspect(pruned.path.model.fit, what = "rsquare"))